Thie notebook explores using BERT for text classification.  Before starting, change the runtime to GPU: Runtime > Change runtime type > Hardware accelerator: GPU.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=fdad4ae03430ab1bd030ac636598085e0903c2ef2b314006e81dbc050947a3f5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import random
import time
from scipy import sparse
from sklearn import linear_model
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from collections import Counter

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on {}".format(device))

Running on cuda


In [ ]:
def read_labels(filename):
    labels={}
    with open(filename, "r", encoding="utf-8-sig") as file:
        for line in file:
            cols = line.split("\t")
            label = cols[0]
            if label not in labels:
                labels[label]=len(labels)
    return labels

In [ ]:
def read_data(filename, labels, max_data_points=None):
    """
    :param filename: the name of the file
    :return: list of tuple ([word index list], label)
    as input for the forward and backward function
    """
    data = []
    data_labels = []
    with open(filename, "r", encoding="utf-8-sig") as file:
        for line in file:
            cols = line.split("\t")
            label = cols[0]
            text = cols[1]

            data.append(text)
            data_labels.append(labels[label])


    # shuffle the data - regression training needs shuffles (see class of Hypothesis testing 2)
    tmp = list(zip(data, data_labels))
    random.shuffle(tmp)
    data, data_labels = zip(*tmp)

    if max_data_points is None:
        return data, data_labels

    return data[:max_data_points], data_labels[:max_data_points]

In [ ]:
labels=read_labels("train600.tsv")
print(labels)

{'NK': 0, 'K': 1}


In [ ]:
train_x, train_y=read_data("train600.tsv", labels, max_data_points=600)

In [ ]:
dev_x, dev_y=read_data("dev200.tsv", labels, max_data_points=200)

In [ ]:
test_x, test_y=read_data("test200.tsv", labels, max_data_points=200)

In [ ]:
print(len(train_x))

600


In [ ]:
def evaluate(model, all_x, all_y):
    model.eval()
    corr = 0.
    total = 0.
    with torch.no_grad():
        idx=0
        for x, y in zip(all_x, all_y):

            idx+=1
            y_preds=model.forward(x)
            for idx, y_pred in enumerate(y_preds):
                prediction=torch.argmax(y_pred)
                if prediction == y[idx]:
                    corr += 1.
                total+=1
    return corr/total

In [ ]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
import time
from tqdm import tqdm
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sklearn.metrics import f1_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Epoch 17, dev accuracy: 0.765


100%|██████████| 75/75 [00:00<00:00, 1129.72it/s]


Epoch 18, dev accuracy: 0.765


100%|██████████| 75/75 [00:00<00:00, 1132.33it/s]

Epoch 19, dev accuracy: 0.765

Best Performing Model achieves dev accuracy of: 0.770
Time: 51.847 seconds ---


In [ ]:
import time
import torch
from sentence_transformers import SentenceTransformer
from torch import nn
from tqdm import tqdm

class SentenceEmbeddingClassifier(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.num_labels = params["label_length"]
        self.fc = nn.Linear(params["embedding_size"], self.num_labels)

    def forward(self, embeddings):
        out = self.fc(embeddings)
        return out

def get_sentence_embedding(sentence_model, sentence):
    return sentence_model.encode(sentence)

def get_batches(all_x, all_y, sentence_model, batch_size=64):
    batches_x, batches_y = [], []
    for i in range(0, len(all_x), batch_size):
        batch_x = []
        for data_point in all_x[i:i+batch_size]:
            response_embedding = get_sentence_embedding(sentence_model, data_point)
            batch_x.append(response_embedding)
        batch_x = torch.tensor(batch_x).to(device)
        batch_y = torch.LongTensor(all_y[i:i+batch_size]).to(device)
        batches_x.append(batch_x)
        batches_y.append(batch_y)
    return batches_x, batches_y

def evaluate(model, batches_x, batches_y):
    model.eval()
    all_predictions = []
    all_true_labels = []

    with torch.no_grad():
        for x, y in zip(batches_x, batches_y):
            outputs = model(x)
            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_true_labels.extend(y.cpu().numpy())

    accuracy = 100 * sum(p == t for p, t in zip(all_predictions, all_true_labels)) / len(all_true_labels)
    f1 = f1_score(all_true_labels, all_predictions, average='weighted')

    return accuracy, f1

def train_and_evaluate(train_x, train_y, dev_x, dev_y, test_x, test_y, labels, embedding_size=768):
    start_time = time.time()

    sentence_model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

    params = {"embedding_size": embedding_size, "label_length": len(labels)}
    classifier = SentenceEmbeddingClassifier(params).to(device)

    train_batches_x, train_batches_y = get_batches(train_x, train_y, sentence_model)
    dev_batches_x, dev_batches_y = get_batches(dev_x, dev_y, sentence_model)
    test_batches_x, test_batches_y = get_batches(test_x, test_y, sentence_model)

    optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-5)
    cross_entropy = nn.CrossEntropyLoss()

    num_epochs = 50
    best_dev_acc = 0.

    for epoch in range(num_epochs):
        classifier.train()

        for x, y in tqdm(list(zip(train_batches_x, train_batches_y))):
            y_pred = classifier(x)
            loss = cross_entropy(y_pred.view(-1, classifier.num_labels), y.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        dev_accuracy, dev_f1 = evaluate(classifier, dev_batches_x, dev_batches_y)
        print(f"Epoch {epoch}, Dev Accuracy: {dev_accuracy:.3f}, Dev F1 Score: {dev_f1:.3f}")
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy

    test_accuracy, test_f1 = evaluate(classifier, test_batches_x, test_batches_y)
    print(f"\nBest Performing Model achieves dev accuracy of: {best_dev_acc:.3f}")
    print(f"Test Accuracy: {test_accuracy:.3f}, Test F1 Score: {test_f1:.3f}")
    print(f"Time: {time.time() - start_time:.3f} seconds ---")

In [ ]:
train_and_evaluate(train_x, train_y, dev_x, dev_y, test_x, test_y, labels)

100%|██████████| 10/10 [00:00<00:00, 612.54it/s]


Epoch 0, Dev Accuracy: 52.500, Dev F1 Score: 0.562


100%|██████████| 10/10 [00:00<00:00, 647.13it/s]


Epoch 1, Dev Accuracy: 52.000, Dev F1 Score: 0.557


100%|██████████| 10/10 [00:00<00:00, 844.40it/s]


...

Epoch 47, Dev Accuracy: 72.000, Dev F1 Score: 0.659


100%|██████████| 10/10 [00:00<00:00, 667.62it/s]


Epoch 48, Dev Accuracy: 73.000, Dev F1 Score: 0.664


100%|██████████| 10/10 [00:00<00:00, 703.73it/s]


Epoch 49, Dev Accuracy: 73.000, Dev F1 Score: 0.664

Best Performing Model achieves dev accuracy of: 73.000
Test Accuracy: 74.500, Test F1 Score: 0.670
Time: 17.650 seconds ---
